# <u>Métodos Supervisados para Regresión - Librería Sklearn y Statsmodels</u>

### Importar paquetes requeridos

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import pylab as pl
import numpy as np
%matplotlib inline


<h2 id="understanding_data">Comprendiendo los datos</h2>

### `FuelConsumption.csv`:
Hemos descargado un conjunto de datos de consumo de combustible, **`FuelConsumption.csv`**, que contiene clasificaciones de consumo de combustible específicas del modelo y emisiones estimadas de dióxido de carbono para nuevos vehículos ligeros para la venta al por menor en Canadá. [Dataset source](http://open.canada.ca/data/en/dataset/98f1a129-f628-4ce4-b24d-6f16bf24dd64)

- **MODELYEAR** e.g. 2014
- **MAKE** e.g. Acura
- **MODEL** e.g. ILX
- **VEHICLE CLASS** e.g. SUV
- **ENGINE SIZE** e.g. 4.7
- **CYLINDERS** e.g 6
- **TRANSMISSION** e.g. A6
- **FUEL CONSUMPTION in CITY(L/100 km)** e.g. 9.9
- **FUEL CONSUMPTION in HWY (L/100 km)** e.g. 8.9
- **FUEL CONSUMPTION COMB (L/100 km)** e.g. 9.2
- **CO2 EMISSIONS (g/km)** e.g. 182   --> low --> 0


<h2 id="reading_data">Leyendo los datos de entrrada</h2>

In [ ]:
# PARA CARGAR DATOS EN EL ENTORNO DE GOOGLE COLAB

# from google.colab import files
# uploaded = files.upload()

# import io
# import pandas as pd
# df = pd.read_csv(io.BytesIO(uploaded['FuelConsumption.csv']),encoding='latin-1', sep = ',')

# # take a look at the dataset
# df.head()

In [ ]:
df = pd.read_csv('data/FuelConsumption.csv',encoding='latin-1', sep = ',')
df.head()

<h2 id="data_exploration">Exploración de datos</h2>
Primero hagamos una exploración descriptiva de nuestros datos.

In [ ]:
df.info()

In [ ]:
# summarize the data
df.describe()

In [ ]:
df.describe(include=object)

Vamos a seleccionar algunas funciones para explorar más:

In [ ]:
cdf = df[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB','CO2EMISSIONS']]
cdf.head(9)

In [ ]:
cdf.hist()

Nosotros podemos graficar cada una de esas características:

In [ ]:
viz = cdf[['CYLINDERS','ENGINESIZE','CO2EMISSIONS','FUELCONSUMPTION_COMB']]
viz.hist()
plt.show()

Ahora, vamos a graficar cada una de las características vs la emisión, para observar cuán lineal es su relación:

In [ ]:
plt.scatter(cdf.FUELCONSUMPTION_COMB, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("FUELCONSUMPTION_COMB")
plt.ylabel("Emission")
plt.show()

In [ ]:
plt.scatter(cdf.ENGINESIZE, cdf.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

#### Creando el conjunto de datos de entrenamiento y prueba
La división Train/Test Split implica dividir el conjunto de datos en conjuntos de entrenamiento y prueba, respectivamente, que son mutuamente excluyentes. Después de lo cual, entrena con el conjunto Train y prueba con el conjunto Test.
Esto proporcionará una evaluación más precisa de la precisión fuera de la muestra porque el conjunto de datos de prueba no es parte del conjunto de datos que se ha utilizado para entrenar los datos. Es más realista para problemas del mundo real.

Esto significa que conocemos el resultado de cada punto de datos en este conjunto de datos, ¡por lo que es genial probarlo! Y dado que estos datos no se han utilizado para entrenar el modelo, el modelo no tiene conocimiento del resultado de estos puntos de datos. Entonces, en esencia, es realmente una prueba fuera de muestra.

Dividamos nuestro conjunto de datos en conjuntos de train/test, el 80% de los datos completos para entrenamiento y el 20% para prueba. Creamos una máscara para seleccionar filas aleatorias usando la función __np.random.rand()__: 

In [ ]:
msk = np.random.rand(len(df)) < 0.8
train = cdf[msk]
test = cdf[~msk]

In [ ]:
len(train)

In [ ]:
len(test)

In [ ]:
207/(860+207)

#### Distribución de la data de entrenamiento

In [ ]:
plt.scatter(train.ENGINESIZE, train.CO2EMISSIONS,  color='blue')
plt.xlabel("Engine size")
plt.ylabel("Emission")
plt.show()

<h2 id="multiple_regression">Modelo de regresión Múltiple</h2>

En realidad, hay múltiples variables que predicen la emisión de Co2. Cuando hay más de una variable independiente presente, el proceso se llama regresión lineal múltiple. Por ejemplo, prediciendo la emisión de CO2 utilizando FUELCONSUMPTION_COMB, EngineSize y Cylinders de automóviles. Lo bueno aquí es que la regresión lineal múltiple es la extensión del modelo de regresión lineal simple.

#### Modelando
Usando el paquete <b>sklearn</b> para modelar la data.

In [ ]:
x = train[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']]
y = train[['CO2EMISSIONS']]

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit (x, y)

In [ ]:
regr.intercept_

In [ ]:
regr.coef_

In [ ]:

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

CO2EMISSIONS = 68.04734992 + 11.90737108 * ENGINESIZE + 7.32087354 * CYLINDERS + 7.32087354 * FUELCONSUMPTION_COMB

Como se mencionó anteriormente, __Coefficient__ and __Intercept__ , son los parámetros de la línea de ajuste. 
Dado que es una regresión lineal múltiple, con 3 parámetros, y sabiendo que los parámetros son la intersección y los coeficientes del hiperplano, <i>sklearn</i> puede estimarlos a partir de nuestros datos. <i>Scikit-learn</i> utiliza el método de mínimos cuadrados ordinarios para resolver este problema.

#### Mínimos Cuadrados Ordinarios/Ordinary Least Squares (OLS)
OLS es un método para estimar los parámetros desconocidos en un modelo de regresión lineal. OLS elige los parámetros de una función lineal de un conjunto de variables explicativas minimizando la suma de los cuadrados de las diferencias entre la variable <i>target</i> dependiente y las predichas por la función lineal. En otras palabras, intenta minimizar la suma de los errores al cuadrado (SSE) o el error al cuadrado medio (MSE) entre la variable objetivo <i>(y)</i> y nuestra salida predicha ($\hat{y}$) sobre todas las muestras en el conjunto de datos.

OLS puede encontrar los mejores parámetros utilizando los siguientes métodos:
   - Resolviendo los parámetros del modelo analíticamente usando ecuaciones de forma cerrada.
   - Usando de un algoritmo de optimización (Descenso de gradiente, Descenso de gradiente estocástico, Método de Newton, etc.)

#### Evaluación
comparamos los valores reales y los valores pronosticados para calcular la precisión de un modelo de regresión. Las métricas de evaluación proporcionan un papel clave en el desarrollo de un modelo, ya que proporcionan información sobre las áreas que requieren mejoras.

Existen diferentes métricas de evaluación del modelo, usamos MSE aquí para calcular la precisión de nuestro modelo en función del conjunto de pruebas: 
<ul>
    <li> Mean absolute error: Error absoluto medio es la media del valor absoluto de los errores. Esta es la métrica más fácil de entender, ya que es solo un error promedio.</li>
    <li> Mean Squared Error (MSE): Es la media del error cuadrado. Es más popular que el error absoluto medio porque el enfoque se orienta más hacia errores grandes. Esto se debe a que el término cuadrado aumenta exponencialmente los errores más grandes en comparación con los más pequeños.</li>
    <li> Root Mean Squared Error (RMSE): Esta es la raíz cuadrada del error cuadrático medio.</li>
    <li> R-squared no es un error, pero es una métrica popular para la precisión de tu modelo. Representa lo cerca que están los datos de la línea de regresión ajustada. Cuanto más alto sea el R cuadrado, mejor se ajustará el modelo a tus datos. La mejor puntuación posible es 1.0 y puede ser negativa (porque el modelo puede ser arbitrariamente peor).</li>
</ul>

In [ ]:
regr.score(x,y) # R2

In [ ]:
y_hat= regr.predict(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])

In [ ]:
y_hat

In [ ]:
y_hat= regr.predict(test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']])
x = test[['ENGINESIZE','CYLINDERS','FUELCONSUMPTION_COMB']]
y = test[['CO2EMISSIONS']]
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat - y)))
print("Residual sum of squares: %.2f"
      % np.mean((y_hat - y) ** 2))

# Explained variance score: 1 is perfect prediction
print('R2-score: %.2f' % regr.score(x, y))

# ¿Podría utilizar las variables categóricas?

Sí, transformándolas en variables dummy

In [ ]:
df.head()

In [ ]:
df.shape

In [ ]:
df.describe(include=object)

Vemos que las variables categóricas que pueden ayudar son FUELTYPE y excepcionalmente VEHICLECLASS pueden contribuir, vamos a probar ambas variables adicionales a las que teníamos:

In [ ]:
# Convertir las variables categóricas en variables ficticias o dummies:

df2 = df.drop(columns = ['MAKE','MODEL','TRANSMISSION'])
df_dummies = pd.get_dummies(df2)

In [ ]:
df_dummies.head()

In [ ]:
df_dummies.shape

In [ ]:
# División en train y test

msk = np.random.rand(len(df_dummies)) < 0.8
train2 = df_dummies[msk]
test2 = df_dummies[~msk]

In [ ]:
x_train = train2.drop(columns = 'CO2EMISSIONS')
y_train = train2[['CO2EMISSIONS']]

x_test = test2.drop(columns = 'CO2EMISSIONS')
y_test = test2[['CO2EMISSIONS']]

In [ ]:
from sklearn import linear_model
regr = linear_model.LinearRegression()
regr.fit (x_train, y_train)

In [ ]:
#haciendo las predicciones
y_hat_train= regr.predict(train2.drop(columns = 'CO2EMISSIONS'))
y_hat_test= regr.predict(test2.drop(columns = 'CO2EMISSIONS'))

In [ ]:
# evaluación de las métricas en train
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat_train - y_train)))
print("Residual sum of squares: %.2f" % np.mean((y_hat_train - y_train) ** 2))
# Explained variance score: 1 is perfect prediction
print('R2-score: %.2f' % regr.score(x_train, y_train))

In [ ]:
# evaluación de las métricas en test
print("Mean absolute error: %.2f" % np.mean(np.absolute(y_hat_test - y_test)))
print("Residual sum of squares: %.2f" % np.mean((y_hat_test - y_test) ** 2))
# Explained variance score: 1 is perfect prediction
print('R2-score: %.2f' % regr.score(x_test, y_test))

In [ ]:

# The coefficients
print ('Coefficients: ', regr.coef_)
print ('Intercept: ',regr.intercept_)

In [ ]:
# modelo de regresión líneal con statsmodels

import statsmodels.api as sm           
import statsmodels.formula.api as smf  

M_R = sm.OLS(y_train,x_train).fit()
M_R.summary()